In [1]:
# !pip install --upgrade torch==2.0.1 --index-url https://download.pytorch.org/whl/cu118
# !pip install -U sentence-transformers
# !pip install rouge bitsandbytes accelerate einops huggingface_hub
# !pip install -q -U trl transformers==4.35.0 git+https://github.com/huggingface/peft.git

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB ? eta 0:00:00a 0:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 20.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89989 sha256=9816cda0adfac053b105a3d0844f0469093fc5d9742cc13d8a7fe98cf80eda40
  Stored in directory: /root/.cache/pip/wheels/27/2c/b6/3ed2983b1b44fe0dea1bb35234b09f2c22fb8ebb308679c922
Successfully built lit
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Create

In [13]:
import re
import pandas as pd

from tqdm import tqdm
from glob import glob
from pathlib import Path
from string import punctuation
from sentence_transformers import SentenceTransformer

from src.preprocess import hybrid_search_topn_diseases, preprocess_corpus_files, preprocess_option, merge_options
from src.index import split_and_embed_corpus
from src.retrieval import retrieve_context
from src.llm import load_llm, predict, is_the_answer_correct, instruct_prompt

corpus_path = Path('/kaggle/input/kalapa-vietmedqa/corpus')
corpus_preproc_path = Path('/kaggle/working/output')
test_path = Path('/kaggle/input/kalapa-vietmedqa/public_test.csv')

# 0. Preprocessing steps.
preprocess_corpus_files(corpus_path, corpus_preproc_path)

test = pd.read_csv(test_path)
for i in range(1, 7):
    test[f'option_{i}_pp'] = test[f'option_{i}'].apply(preprocess_option)
    assert len(test[test[f'option_{i}'] == '']) == 0, i
    test.drop(columns=[f'option_{i}'], inplace=True)
test['options'] = test.apply(merge_options, axis=1)

# 1. Indexing (split and embed texts).
corpus_glob = str(corpus_preproc_path/'*')
disease_files = glob(corpus_glob)
diseases = [Path(p).stem.lower().replace('hội chứng', '').replace(
    'bệnh', '').strip().split(' ') for p in glob(corpus_glob)]
questions = [re.sub(f"[{punctuation}]", "", q).lower().split()
             for q in test['question'].tolist()]
answers = [re.sub(f"[{punctuation}]", "", q).lower().split()
           for q in test['options'].tolist()]
ids = test['id'].tolist()

# Text => chunks of text => embeddings.
doc_chunks, doc_chunks_emb = split_and_embed_corpus(
    disease_files,
    model='distiluse-base-multilingual-cased-v2'
)

# 2. Retrieval steps.
# Reduce the number of disease files to look at for each question.
topn_diseases_dict = hybrid_search_topn_diseases(
    disease_files, diseases, questions, answers, ids, n=5
)

# Start retrieving.
embedding_model = SentenceTransformer(
    'distiluse-base-multilingual-cased-v2')
retrieved_context_dict = {}
for index, row in tqdm(test.iterrows(), total=test.shape[0]):
    doc_id = row.id
    query = row.question + '\n' + row.options

    files_to_look_at = topn_diseases_dict[doc_id]['topn_disease']

    # print('index=', doc_id)
    # print(query, '\n')
    # print(files_to_look_at, '\n')
    # print('#' * 5)
    retrieved_context = retrieve_context(
        query, files_to_look_at, doc_chunks, doc_chunks_emb, embedding_model)
    retrieved_context_dict[doc_id] = retrieved_context

# 3. Augmented LLM.
llm, tokenizer = load_llm()

# Testing on only 1 document.
print(predict(test.iloc[0], llm, tokenizer,
      retrieved_context_dict, correct_str='1', incorrect_str='0'))

# You can run the rest here if you want.

  0%|          | 0/603 [00:00<?, ?it/s]/kaggle/working/src/index.py:13: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 13 of the file /kaggle/working/src/index.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(split)
Token indices sequence length is longer than the specified maximum sequence length for this model (945 > 512). Running this sequence through the model will result in indexing errors
100%|██████████| 603/603 [01:39<00:00,  6.06it/s]


Total chunks: 19083


  1%|          | 1/100 [00:00<00:12,  7.98it/s]

Question: hương đang mang thai và lo lắng mình có thể gặp phải rau tiền đạo hương có thể kiểm tra phát hiện bệnh này từ tuần thứ mấy của thai kỳ
Top-n from question: ['Rau tiền đạo', 'Nhau tiền đạo', 'Thai lưu', 'Mang thai ngoài tử cung', 'Tăng huyết áp thai kỳ']




  2%|▏         | 2/100 [00:00<00:12,  7.86it/s]

...

Question: u não là tình trạng các khối u hình thành trong sọ não đe dọa tính mạng người bệnh u não thường có 2 loại là lành tính và ác tính đâu là điểm chung của chúng
Top-n from question: ['U não', 'U não lành tính', 'U thận lành tính', 'Bướu cổ lành tính', 'U xương']




100%|██████████| 100/100 [00:09<00:00, 10.97it/s]

Question: vắc xin là chế phẩm có tính kháng nguyên có nguồn gốc từ vi sinh vật gây bệnh hoặc vi sinh vật có cấu trúc kháng nguyên giống vi sinh vật gây bệnh đã được bào chế đảm bảo độ an toàn cần thiết làm cho cơ thể tự tạo ra tình trạng miễn dịch chống lại tác nhân gây bệnh vắc xin có thể phòng ngừa được bệnh nào dưới đây
Top-n from question: ['Bệnh tự miễn', 'Sinh non', 'Vô sinh', 'Chống mặt', 'Vô sinh nguyên phát']





100%|██████████| 100/100 [00:01<00:00, 86.31it/s]


Iteration #1
Bối cảnh: Bao nhiêu tuần thì biết được rau tiền đạo? Bác sĩ Nguyễn Đức Hinh cho biết, hầu hết các trường hợp rau tiền đạo có thể được phát hiện sớm từ tuần 20 của thai kỳ nhờ vào phương pháp siêu âm thai. Trong quá trình siêu âm, với sự hỗ trợ của nhiều thiết bị hiện đại, bác sĩ có thể quan sát được bánh rau bám vào vị trí nào ở tử cung ( thân, đáy, mặt trước, mặt sau, bên trái, bên phải, bám thấp, bán trung tâm hoặc trung tâm ). Chẩn đoán rau tiền đạo.mổ lấy thai khẩn cấp để cứu sống cả thai phụ lẫn thai nhi. Nếu thai nhi chưa đủ ngày đủ tháng sẽ bị sinh non, nguy cơ bị suy hô hấp sau sinh nếu không được can thiệp hỗ trợ. Thêm vào đó, bánh rau nằm ở vị trí dưới phần tử cung sẽ khiến thai nhi khó xoay đầu xuống, gây bất lợi cho đường ra của thai nhi lúc sinh nở. Thai phụ có thể phải sinh ngôi thai ngược ( ngôi ngang hoặc ngôi mông )..Phòng ngừa rau tiền đạo Rau tiền đạo là một trong những biến chứng thai kỳ nguy hiểm, đe dọa tính mạng thai phụ lẫn thai nhi. Chính vì thế, b

**You could see that if we change the `choice`, the model answer differently**
* "0" means the `choice` is not correct and "1" means otherwise.

In [16]:
is_the_answer_correct(
    llm,
    tokenizer,
    instruct_prompt.format(
        # Eng sub: "Diarrhea can be cured by taking medicine."
        context='Tiêu chảy thì uống thuốc là hết.',
        # Eng sub: "What do we do when we got diarrhea?"
        question='Tiêu chảy thì làm gì?',
        choice='Uống nước.'  # Eng sub: "Dink water."
    ))

('0', array([0.8620035 , 0.13799646], dtype=float32))

In [17]:
is_the_answer_correct(
    llm,
    tokenizer,
    instruct_prompt.format(
        context='Tiêu chảy thì uống thuốc là hết.',
        question='Tiêu chảy thì làm gì?',
        choice='Uống thuốc.'))  # Eng sub: "Taking medicine."

('1', array([0.36026397, 0.63973606], dtype=float32))